In [4]:
import os
import openai
from openai import OpenAI
import requests, json
OLLAMA_URL = "http://localhost:11434/api/generate"

In [6]:
"""
LLM plays Minesweeper N times
"""

from __future__ import annotations
import random, json, csv, time
from dataclasses import dataclass, asdict
from pathlib import Path
from typing import List, Tuple, Literal

import openai
import pandas as pd
from tqdm import trange

#  A. Python Minesweeper engine 
class MinesweeperBoard:
    NEIGHB = (
        (-1, -1), (-1,  0), (-1,  1),
        ( 0, -1),          ( 0,  1),
        ( 1, -1), ( 1,  0), ( 1,  1),
    )

    def __init__(self, rows: int, cols: int, mines: int, rng=random):
        if not (0 < mines < rows * cols):
            raise ValueError("mines must be in (0, rows*cols)")
        self.R, self.C, self.M = rows, cols, mines
        self._rng = rng

        # actual content (hidden from player)
        self._grid: List[List[str]] = [["0"] * cols for _ in range(rows)]
        self._lay_mines()
        self._fill_numbers()

        # player’s view: "□" unknown, "⚑" flagged, "0-8" open numbers
        self.view  = [["□"] * cols for _ in range(rows)]
        self.opened = 0
        self.finished, self.win = False, False

    # internal helpers --------------------------------------------------------
    def _lay_mines(self):
        cells = self._rng.sample(range(self.R * self.C), self.M)
        for idx in cells:
            r, c = divmod(idx, self.C)
            self._grid[r][c] = "*"

    def _fill_numbers(self):
        def around(r, c):  # count adjacent mines
            return sum(
                1 for dr, dc in self.NEIGHB
                if 0 <= r+dr < self.R and 0 <= c+dc < self.C
                and self._grid[r+dr][c+dc] == "*"
            )
        for r in range(self.R):
            for c in range(self.C):
                if self._grid[r][c] == "*": continue
                self._grid[r][c] = str(around(r, c))

    # public API --------------------------------------------------------------
    def step(self, r: int, c: int, action: Literal["open", "flag"]) -> str:
        """Returns one of {"ok", "hit-mine", "invalid", "already"}"""
        if self.finished:            return "invalid"
        if not (0 <= r < self.R and 0 <= c < self.C): return "invalid"

        cell = self.view[r][c]

        if action == "flag":
            if cell in ("□", "⚑"):
                self.view[r][c] = "⚑" if cell == "□" else "□"
                return "ok"
            return "already"

        # action == "open"
        if cell != "□":              return "already"
        if self._grid[r][c] == "*":
            self.view[r][c] = "*"
            self.finished = True
            self.win = False
            return "hit-mine"
        self._flood_fill(r, c)
        if self.opened == self.R * self.C - self.M:
            self.finished = True
            self.win = True
        return "ok"

    def _flood_fill(self, r, c, seen=None):
        if seen is None: seen = set()
        if (r, c) in seen or self.view[r][c] != "□": return
        seen.add((r, c))
        self.view[r][c] = self._grid[r][c]
        self.opened += 1
        if self._grid[r][c] == "0":
            for dr, dc in self.NEIGHB:
                nr, nc = r+dr, c+dc
                if 0 <= nr < self.R and 0 <= nc < self.C:
                    self._flood_fill(nr, nc, seen)

    # formats -----------------------------------------------------------------
    def player_view_str(self) -> str:
        return "\n".join(" ".join(row) for row in self.view)

    def reveal_str(self) -> str:
        """Original grid with mines visible – for logging/debug."""
        return "\n".join(" ".join(row) for row in self._grid)

# Dataclass for a single game result
@dataclass
class GameRecord:
    game_id   : int
    rows      : int
    cols      : int
    mines     : int
    moves     : int
    outcome   : Literal["win", "loss"]
    duration_s: float
    rng_seed  : int



# Run a batch of games
def run_experiment(n_games=5,
                   rows=10, cols=10, mines=15,
                   temperature=0.0):
    records: List[GameRecord] = []
    for gid in trange(1, n_games + 1, desc="Games"):
        seed = random.randrange(2**32)
        rec = play_one(gid, rows, cols, mines, temperature, seed=seed)
        records.append(rec)

        # write incrementally (safe if kernel crashes mid-way)
        with open("results.csv", "a", newline="") as f:
            w = csv.DictWriter(f, fieldnames=GameRecord.__annotations__.keys())
            if f.tell() == 0: w.writeheader()
            w.writerow(asdict(rec))

    # Summary
    df = pd.DataFrame([asdict(r) for r in records])
    print("\nBatch summary:")
    print(df.groupby("outcome").size())
    print("Mean moves:", df["moves"].mean())

In [10]:
# ────────────────────────────────────────────────────────────────
#  Probability helper – fast heuristic, no heavy CSP needed
# ────────────────────────────────────────────────────────────────
def compute_probabilities(board) -> dict[tuple[int, int], float]:
    """
    Return P(cell has a mine) for every still-unknown square.

    Heuristic:
      • For every opened number N with U unknown neighbours and F flags,
        each of those U squares carries base-prob  (N-F)/U.
      • A cell may be adjacent to several numbers – average their values.
      • Non-frontier unknowns get a 'global' probability:
            (remaining_mines) / (remaining_unknown_cells).
    Good enough for >80 % wins; you can swap in a Monte-Carlo sampler later
    without touching the solver loop.
    """
    frontier_probs: dict[tuple[int, int], list[float]] = {}
    remaining_mines = board.M - sum(
        1 for r in range(board.R) for c in range(board.C) if board.view[r][c] == "⚑"
    )

    # Pass 1 – collect constraints from every opened number
    for r in range(board.R):
        for c in range(board.C):
            ch = board.view[r][c]
            if ch.isdigit():
                N = int(ch)
                flags, unknowns = 0, []
                for dr, dc in board.NEIGHB:
                    nr, nc = r + dr, c + dc
                    if 0 <= nr < board.R and 0 <= nc < board.C:
                        if board.view[nr][nc] == "⚑":
                            flags += 1
                        elif board.view[nr][nc] == "□":
                            unknowns.append((nr, nc))
                if unknowns:
                    p = (N - flags) / len(unknowns)
                    for cell in unknowns:
                        frontier_probs.setdefault(cell, []).append(p)

    # Pass 2 – average multiple contributions
    probs: dict[tuple[int, int], float] = {}
    for cell, plist in frontier_probs.items():
        probs[cell] = sum(plist) / len(plist)

    # Non-frontier unknowns (“deep fog” squares)
    unknown_cells = [
        (r, c)
        for r in range(board.R)
        for c in range(board.C)
        if board.view[r][c] == "□"
    ]
    global_p = remaining_mines / len(unknown_cells) if unknown_cells else 0.0
    for cell in unknown_cells:
        probs.setdefault(cell, global_p)

    return probs


def play_one(game_id: int,
             rows=9, cols=9, mines=10,
             temperature=0.0, max_moves=500, seed=None):
    rng   = random.Random(seed)
    board = MinesweeperBoard(rows, cols, mines, rng=rng)
    start = time.time()
    moves = 0

    while not board.finished and moves < max_moves:
        # 1.  Run classical deterministic rules until nothing changes
        changed = True
        while changed and not board.finished:
            changed = deterministic_step(board)  # <-- implement or reuse
            moves  += int(changed)

        if board.finished: break  # won through pure logic

        # 2.  Probability pass
        probs   = compute_probabilities(board)
        min_p   = min(probs.values())
        best    = [cell for cell, p in probs.items() if abs(p - min_p) < 1e-9]

        if len(best) == 1:
            r, c = best[0]          # clear favourite – open directly
        else:
            r, c, _ = query_llm(board, best, probs, temperature)

        board.step(r, c, "open")
        moves += 1

    duration = time.time() - start
    return GameRecord(
        game_id, rows, cols, mines, moves,
        "win" if board.win else "loss",
        round(duration, 3), seed or 0
    )


def deterministic_step(board) -> bool:
    """
    Apply the standard single-point rule once.
    Return True if *anything* was opened or flagged.
    """
    for r in range(board.R):
        for c in range(board.C):
            ch = board.view[r][c]
            if ch.isdigit() and ch != "0":
                n = int(ch)
                unknowns, flags = [], 0
                for dr, dc in board.NEIGHB:
                    nr, nc = r + dr, c + dc
                    if 0 <= nr < board.R and 0 <= nc < board.C:
                        v = board.view[nr][nc]
                        if v == "□":
                            unknowns.append((nr, nc))
                        elif v == "⚑":
                            flags += 1

                if unknowns and n - flags == len(unknowns):
                    # all unknowns are mines → flag them
                    for cell in unknowns:
                        board.step(*cell, action="flag")
                    return True
                if unknowns and flags == n:
                    # all other unknowns are safe → open them
                    for cell in unknowns:
                        board.step(*cell, action="open")
                    return True
    return False

In [ ]:
def query_llm(board, candidate_cells, probs, temperature=0.0):
    prob_table = "\n".join(
        f"({r},{c})  P≈{probs[(r,c)]:.3f}" for (r, c) in candidate_cells
    )

    prompt = f"""
You are playing Minesweeper. The deterministic solver found several squares
with the *lowest current mine probability*. Pick ONE to open next.

Respond ONLY with JSON like:
{{"row":R,"col":C,"action":"open"}}

Tied candidates and their probabilities:
{prob_table}

Full board (□ = unknown, ⚑ = flag):
{board.player_view_str()}
"""

    # Send request
    resp = requests.post(
        OLLAMA_URL,
        json={
            "model": "llama3:8b",  # Use the correct model name
            "prompt": prompt.strip(),
            "temperature": temperature,
            "stream": True  # default anyway, but keeping it explicit
        },
        stream=True  # Important to handle streamed lines
    )

    if resp.status_code != 200:
        print("Error:", resp.status_code, resp.text)
        r, c = candidate_cells[0]
        return r, c, "open"

    # Collect all parts of streamed response
    full_response = ""
    for line in resp.iter_lines():
        if line:
            try:
                json_data = json.loads(line.decode('utf-8'))
                full_response += json_data.get("response", "")
            except Exception as e:
                print("Stream parse error:", e)
                continue

    #print("Model reply:", full_response)

    # Try to extract JSON from final message
    try:
        move = json.loads(full_response.strip().split("```")[-1] if "```" in full_response else full_response)
        return int(move["row"]), int(move["col"]), "open"
    except Exception as e:
        #print("JSON decode fallback:", e)
        r, c = candidate_cells[0]
        return r, c, "open"


In [22]:
if __name__ == "__main__":
    run_experiment(n_games=5, rows=9, cols=9, mines=10, temperature=0.2)

Games:   0%|                                                | 0/5 [00:00<?, ?it/s]

Model reply: {"row":0,"col":0,"action":"open"}
Model reply: {"row":0,"col":2,"action":"open"}


Games:  20%|████████                                | 1/5 [00:27<01:49, 27.46s/it]

Model reply: {"row":0,"col":4,"action":"open"}
Model reply: {"row":0,"col":0,"action":"open"}
Model reply: {"row":0,"col":3,"action":"open"}
Model reply: {"row":7,"col":5,"action":"open"}


Games:  40%|████████████████                        | 2/5 [00:48<01:10, 23.59s/it]

...

Model reply: {"row":3,"col":0,"action":"open"}
Model reply: {"row":0,"col":0,"action":"open"}


Games:  80%|████████████████████████████████        | 4/5 [01:20<00:18, 18.59s/it]

Model reply: {"row":0,"col":2,"action":"open"}
Model reply: {"row":0,"col":0,"action":"open"}


Games: 100%|████████████████████████████████████████| 5/5 [01:31<00:00, 18.30s/it]

Model reply: {"row":7,"col":0,"action":"open"}

Batch summary:
outcome
loss    2
win     3
dtype: int64
Mean moves: 16.6


In [23]:
if __name__ == "__main__":
    run_experiment(n_games=10, rows=15, cols=15, mines=15, temperature=0.2)

Games:  10%|███▉                                   | 1/10 [00:27<04:08, 27.61s/it]

Model reply: {"row": 0, "col": 0, "action": "open"}
Model reply: {"row":0,"col":0,"action":"open"}
Model reply: {"row":0,"col":0,"action":"open"}
Model reply: {"row":0,"col":0,"action":"open"}
Model reply: {"row":0,"col":0,"action":"open"}
Model reply: {"row":0,"col":0,"action":"open"}
Model reply: {"row":0,"col":0,"action":"open"}
Model reply: {"row":0,"col":0,"action":"open"}
Model reply: {"row": 0, "col": 0, "action": "open"}
Model reply: {"row":0,"col":0,"action":"open"}
Model reply: {"row":0,"col":0,"action":"open"}
Model reply: {"row":0,"col":0,"action":"open"}
Model reply: {"row":0,"col":0,"action":"open"}
Model reply: {"row":0,"col":0,"action":"open"}
Model reply: {"row":0,"col":0,"action":"open"}
Model reply: {"row":0,"col":0,"action":"open"}
Model reply: {"row":0,"col":0,"action":"open"}
Model reply: {"row": 0, "col": 0, "action": "open"}
Model reply: {"row":0,"col":0,"action":"open"}
Model reply: {"row": 0, "col": 0, "action": "open"}
Model reply: {"row": 0, "col": 0, "actio

Games:  20%|███████▊                               | 2/10 [01:49<07:55, 59.49s/it]

Model reply: {"row":0,"col":4,"action":"open"}


Games:  30%|███████████▋                           | 3/10 [02:14<05:07, 43.89s/it]

...

Games:  90%|███████████████████████████████████    | 9/10 [02:24<00:05,  5.27s/it]

Model reply: {"row":0,"col":0,"action":"open"}


Games: 100%|██████████████████████████████████████| 10/10 [02:27<00:00, 14.72s/it]

Model reply: {"row": 0, "col": 0, "action": "open"}

Batch summary:


outcome
loss    1
win     9
dtype: int64
Mean moves: 26.4


In [24]:
if __name__ == "__main__":
    run_experiment(n_games=25, rows=20, cols=20, mines=30, temperature=0.2)

Games:   4%|█▌                                     | 1/25 [01:07<26:52, 67.20s/it]

Model reply: A Minesweeper puzzle!

The deterministic solver has found several squares with the *lowest current mine probability*, which are marked with `*`.

Now it's my turn to play. I'll start by flagging some suspicious squares.

Here are my first moves:

Flag: (19,13)
Flag: (17,7)
Flag: (11,5)

And here's the updated board:

□ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □
□ □ ⚑ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □
□ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □
□ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □
□ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □
□ □ ⚑ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □
□ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □
□ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □
□ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □

What's next?
JSON decode fallback: Expecting value: line 1 column 1 (char 0)
Model reply: It looks like you're playing Minesweeper!

The deterministic solver has found several squares with the lowest current mine probability, which is indicated by the * symbol. These squares are likely to have mines nearby.

A

Games:   8%|██▉                                 | 2/25 [05:33<1:10:45, 184.60s/it]

Model reply: It looks like you're playing Minesweeper! You've got a full board with many unknown squares (□) and some flagged squares (⚑). The deterministic solver has told you that there are several squares with the lowest current mine count.

As you look at the board, you notice that there aren't any obvious patterns or clusters of mines. However, you do see a few squares that have a relatively low mine count. Those might be good places to start exploring!

What would you like to do next? Would you like to:

1. Flag a square based on its current mine count?
2. Reveal the contents of an unknown square?
3. Use a different strategy or tool to help you solve the puzzle?

Let me know your move!
JSON decode fallback: Expecting value: line 1 column 1 (char 0)


Games:  12%|████▌                                 | 3/25 [06:27<45:39, 124.54s/it]

...

Model reply: It looks like you're playing Minesweeper!

You've got a pretty big board with lots of squares, and the deterministic solver has already found some squares with a low probability of containing a mine. That's helpful to know!

Now it's your turn to take a closer look at each square and try to figure out which ones might be hiding mines. You can use the numbers to help you out - they give you an idea of how many adjacent squares contain mines.

Which square would you like to start with?
JSON decode fallback: Expecting value: line 1 column 1 (char 0)


Games:  96%|█████████████████████████████████▌ | 24/25 [1:18:10<03:05, 185.77s/it]

Model reply: You're playing Minesweeper!

I'll help you by pointing out that the deterministic solver has identified several squares with the lowest current mine probability (0.075). These are:

* (16, 15) ⚑
* (18, 11) ⚑
* (19, 17) ⚑

You can flag these squares to avoid them.

Now it's your turn! Which square would you like to choose next? Remember, you can either reveal a cell or place a flag on an unknown cell.
JSON decode fallback: Expecting value: line 1 column 1 (char 0)


Games: 100%|███████████████████████████████████| 25/25 [1:18:52<00:00, 189.30s/it]

Model reply: It looks like you're trying to solve a Minesweeper puzzle! 🤔

The deterministic solver found some squares with the *lowest current mine probability*. These are marked with `*`. Now it's your turn to play and decide which squares to flag (`⚑`) or not.

To make a move, simply type the number of the square you want to flag (e.g., "1" for the top-left corner). You can also ask for hints or request more information about specific squares. 😊
JSON decode fallback: Expecting value: line 1 column 1 (char 0)

Batch summary:
outcome
loss     4
win     21
dtype: int64
Mean moves: 47.28
